In [1]:
import numpy as np
import random, math
import copy

Dimension = 4
qubits = 2
A = np.array([[-9, -2,-2,9], [-3, 10,-3,-3], [-8,6,-9,-8], [-8,8,10,9]]) # Row vectors
b = np.array([-29, 19, 30, -19]) # Column vectors
# x1 = q1 + 2q2 - 4q3

QM = np.zeros(((qubits+1)*Dimension, (qubits+1)*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        cef1 = pow(2,2*qubits)*pow(A[k][i],2)
        cef2 = pow(2,qubits+1)*A[k][i]*b[k]
        po2 = (qubits+1)*i + qubits 
        QM[po2][po2] = QM[po2][po2] + cef1 + cef2
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = (qubits+1)*i + l
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l in range (qubits):
            qcef = pow(2, l+qubits+1)*pow(A[k][i],2)
            po3 = (qubits+1)*i + l
            po4 = (qubits+1)*i + qubits 
            QM[po3][po4] = QM[po3][po4] - qcef
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits+1)*i + l1
                po2 = (qubits+1)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                
### Second quadratic term ###                 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            qcef = pow(2, 2*qubits+1)*A[k][i]*A[k][j]
            po3 = (qubits+1)*i + qubits
            po4 = (qubits+1)*j + qubits
            QM[po3][po4] = QM[po3][po4] + qcef
            for l in range (qubits):
                qcef = pow(2, l+qubits+1)*A[k][i]*A[k][j]
                po5 = (qubits+1)*i + qubits 
                po6 = (qubits+1)*j + l
                QM[po5][po6] = QM[po5][po6] - qcef
                po7 = (qubits+1)*i + l
                po8 = (qubits+1)*j + qubits
                QM[po7][po8] = QM[po7][po8] - qcef
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = (qubits+1)*i + l1
                    po2 = (qubits+1)*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef


# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range((qubits+1)*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",(qubits+1)*Dimension,"','q",(qubits+1)*Dimension,"'):",format(QM[(qubits+1)*Dimension-1][(qubits+1)*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range((qubits+1)*Dimension-1):
    for j in range(i+1,(qubits+1)*Dimension):
        qdrt1 = i + 1
        qdrt2 = j + 1
        if i == (qubits+1)*Dimension-2 and j == (qubits+1)*Dimension-1:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
        else:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[  -14.   872. -1744.  -248.  -496.   992.    38.    76.  -152.  -160.
   -320.   640.]
 [    0.   408. -3488.  -496.  -992.  1984.    76.   152.  -304.  -320.
   -640.  1280.]
 [    0.     0.  4416.   992.  1984. -3968.  -152.  -304.   608.   640.
   1280. -2560.]
 [    0.     0.     0.  -348.   816. -1632.     0.     0.     0.   -48.
    -96.   192.]
 [    0.     0.     0.     0.  -288. -3264.     0.     0.     0.   -96.
   -192.   384.]
 [    0.     0.     0.     0.     0.  5472.     0.     0.     0.   192.
    384.  -768.]
 [    0.     0.     0.     0.     0.     0.  1112.   776. -1552.   306.
    612. -1224.]
 [    0.     0.     0.     0.     0.     0.     0.  2612. -3104.   612.
   1224. -2448.]
 [    0.     0.     0.     0.     0.     0.     0.     0.  -568. -1224.
  -2448.  4896.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.  1693.
    940. -1880.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   3856. -3760.]
 [    0

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-14.0, ('q2','q2'):408.0, ('q3','q3'):4416.0, ('q4','q4'):-348.0, ('q5','q5'):-288.0, ('q6','q6'):5472.0, ('q7','q7'):1112.0, ('q8','q8'):2612.0, ('q9','q9'):-568.0, ('q10','q10'):1693.0, ('q11','q11'):3856.0, ('q12','q12'):-2072.0}

quadratic = {('q1','q2'):872.0, ('q1','q3'):-1744.0, ('q1','q4'):-248.0, ('q1','q5'):-496.0, ('q1','q6'):992.0, ('q1','q7'):38.0, ('q1','q8'):76.0, ('q1','q9'):-152.0, ('q1','q10'):-160.0, ('q1','q11'):-320.0, ('q1','q12'):640.0, ('q2','q3'):-3488.0, ('q2','q4'):-496.0, ('q2','q5'):-992.0, ('q2','q6'):1984.0, ('q2','q7'):76.0, ('q2','q8'):152.0, ('q2','q9'):-304.0, ('q2','q10'):-320.0, ('q2','q11'):-640.0, ('q2','q12'):1280.0, ('q3','q4'):992.0, ('q3','q5'):1984.0, ('q3','q6'):-3968.0, ('q3','q7'):-152.0, ('q3','q8'):-304.0, ('q3','q9'):608.0, ('q3','q10'):640.0, ('q3','q11'):1280.0, ('q3','q12'):-2560.0, ('q4','q5'):816.0, ('q4','q6'):-1632.0, ('q4','q7'):0.0, ('q4','q8'):0.0, ('q4','q9'):0.0, ('q4','q10'):-48.0, ('q4','q11'):-96.0, ('q4','q12'):192.0, ('q5','q6'):-3264.0, ('q5','q7'):0.0, ('q5','q8'):0.0, ('q5','q9'):0.0, ('q5','q10'):-96.0, ('q5','q11'):-192.0, ('q5','q12'):384.0, ('q6','q7'):0.0, ('q6','q8'):0.0, ('q6','q9'):0.0, ('q6','q10'):192.0, ('q6','q11'):384.0, ('q6','q12'):-768.0, ('q7','q8'):776.0, ('q7','q9'):-1552.0, ('q7','q10'):306.0, ('q7','q11'):612.0, ('q7','q12'):-1224.0, ('q8','q9'):-3104.0, ('q8','q10'):612.0, ('q8','q11'):1224.0, ('q8','q12'):-2448.0, ('q9','q10'):-1224.0, ('q9','q11'):-2448.0, ('q9','q12'):4896.0, ('q10','q11'):940.0, ('q10','q12'):-1880.0, ('q11','q12'):-3760.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. chain_b.
0    0   1   0   1  0  0  1  0  0  0  0  0 -2463.0       5      0.0
1    1   0   0   1  1  1  0  0  0  1  0  0 -2412.0       8      0.0
2    1   0   1   1  0  0  0  1  0  1  1  1 -2412.0      16      0.0
3    0   0   1   1  0  0  1  0  0  1  1  1 -2340.0      24      0.0
4    0   0   0   1  0  0  1  0  0  1  0  0 -2340.0       7      0.0
5    1   0   0   1  1  1  1  0  0  1  0  0 -2320.0       6      0.0
6    1   0   1   1  0  0  1  0  0  1  1  1 -2320.0       5      0.0
7    1   1   0   1  0  0  0  1  0  0  0  0 -2289.0       3      0.0
8    1   1   0   1  1  1  0  0  0  0  0  0 -2289.0       7      0.0
9    1   1   1   1  0  0  0  1  0  0  1  1 -2289.0       5      0.0
10   0   1   1   1  1  0  0  1  0  0  1  1 -2279.0       8      0.0
11   0   1   0   1  0  0  1  0  0  1  0  0 -2269.0       2      0.0
12   0   1   0   1  0  0  1  0  0  1  1  1 -2269.0      18      0.0
13   1   0   0   1  1  1  0  0  0  0  0  0 -2262